In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.config import list_physical_devices
from tensorflow.image import ResizeMethod
from tensorflow.image import resize_with_pad

import utils

In [2]:
list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
trainX, trainY, testX, testY = utils.load_dataset()

In [4]:
trainX.shape

(50000, 32, 32, 3)

In [5]:
def transfer_densenet():
    inputs = keras.Input(shape=(32,32,3))
    upscale = keras.layers.Lambda(lambda x: resize_with_pad(x, 160, 160, method=ResizeMethod.BILINEAR))(inputs)


    base_model = keras.applications.DenseNet121(include_top=False, weights='imagenet', input_tensor=upscale, pooling='max')
    base_model.trainable=False
    
    x = base_model.output

    x = layers.Flatten()(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(256, activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.3)(x)

    outputs = layers.Dense(10, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="transfer_densenet")
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
    return model

In [6]:
callback = keras.callbacks.ModelCheckpoint('models/transfer_densenet', monitor='val_loss', save_best_only=True)

In [7]:
model = transfer_densenet()
history = model.fit(x=trainX, y=trainY, batch_size=512, epochs=50, callbacks=callback, validation_data=(testX, testY))

Epoch 1/50
98/98 [==============================] - 307s 2s/step - loss: 1.5841 - categorical_accuracy: 0.4686 - auc: 0.8372 - val_loss: 0.7041 - val_categorical_accuracy: 0.7636 - val_auc: 0.9708
INFO:tensorflow:Assets written to: models/transfer_resnet/assets
Epoch 2/50
98/98 [==============================] - 25s 256ms/step - loss: 0.7222 - categorical_accuracy: 0.7579 - auc: 0.9688 - val_loss: 0.5830 - val_categorical_accuracy: 0.8018 - val_auc: 0.9796
INFO:tensorflow:Assets written to: models/transfer_resnet/assets
Epoch 3/50
98/98 [==============================] - 25s 256ms/step - loss: 0.6011 - categorical_accuracy: 0.8012 - auc: 0.9776 - val_loss: 0.5361 - val_categorical_accuracy: 0.8204 - val_auc: 0.9826
INFO:tensorflow:Assets written to: models/transfer_resnet/assets
Epoch 4/50
98/98 [==============================] - 25s 258ms/step - loss: 0.5419 - categorical_accuracy: 0.8220 - auc: 0.9814 - val_loss: 0.5125 - val_categorical_accuracy: 0.8272 - val_auc: 0.9840
INFO:tensor

In [8]:
best_model = keras.models.load_model('models/transfer_densenet')

In [9]:
best_model.evaluate(x=testX, y=testY)

313/313 [==============================] - 278s 454ms/step - loss: 0.4763 - categorical_accuracy: 0.8411 - auc: 0.9849


[0.47633862495422363, 0.8410999774932861, 0.9849041700363159]

In [ ]:
#architecture: https://medium.com/swlh/hands-on-the-cifar-10-dataset-with-transfer-learning-2e768fd6c318